<a href="https://colab.research.google.com/github/fpseverino/progetto-ml-ai/blob/main/Severino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import copy
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingGridSearchCV, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

# Classe Task
Viene definita una classe `Task`, che viene inizializzata a partire da un `DataFrame` e ha come attributi le suddivisioni delle feature e delle label per il training e il testing. L'inizializzatore accetta anche un parametro booleano che attiva o meno la feature selection.

La classe contiene anche un metodo per scalare le feature con uno specifico scaler.

In [9]:
class Task:
  def __init__(self, df, select = False):
    X = df.drop("Label", axis = 1)
    X = X.drop("Task", axis = 1)
    X = X.drop("Id", axis = 1)
    y = df["Label"]

    if select:
      X = SelectKBest(f_classif, k = 100).fit_transform(X, y)

    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

  def scale(self, sc):
    sc.fit(self.X_train)
    self.X_train = sc.transform(self.X_train)
    self.X_test = sc.transform(self.X_test)

Di seguito vengono letti i `csv` dei Task già ripuliti e vengono salvati come oggetti `Task` all'interno dell'array `tasks`.

In [10]:
tasks = []
for i in range(11, 22):
  df = pd.read_csv("Task_{}_clean.csv".format(i))
  task = Task(df, select = True)
  tasks.append(task)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [2] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [8] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [2] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/py

Di seguito viene calcolata la moda delle lable del set di testing e viene definita una funzione per calcolarne la somiglianza con i risultati delle predizioni.

In [11]:
y_tests = []
for task in tasks:
  y_tests.append(copy.deepcopy(task.y_test))

y_test_mode, y_test_count = stats.mode(y_tests)

def similarity(predictions_mode):
  if len(y_test_mode) != len(predictions_mode):
    return 0.0

  correct = 0
  for i in range(len(predictions_mode)):
    if y_test_mode[i] == predictions_mode[i]:
      correct += 1

  return correct / len(predictions_mode)

# Funzione `test_boosting`
Dopo aver eventualmente scalato i `Task` mediante uno scaler, viene eseguito, task per task, la cross validation, poi un fit del classificatore, e vengono salvati gli score e le predizioni. Di tutti i risultati verrà stampata la media attraverso tutti i Task.

In [12]:
def test_boosting(clf, sc = None):
  test_tasks = copy.deepcopy(tasks)

  if sc is not None:
    for task in test_tasks:
      task.scale(sc)

  cross_val_scores = []
  scores = []
  predictions = []

  for task in test_tasks:
    cross_val_scores.append(cross_val_score(clf, task.X_train, task.y_train, cv = 4, n_jobs = -1).mean())
    clf.fit(task.X_train, task.y_train)
    scores.append(clf.score(task.X_test, task.y_test))
    predictions.append(clf.predict(task.X_test))

  print("Results{}:".format("" if sc is None else " (scaled tasks)"))
  print(" - Mean cross_val_score: {:.2%}".format(np.mean(cross_val_scores)))
  print(" - Mean score:           {:.2%}".format(np.mean(scores)))

  mode, count = stats.mode(predictions)
  print(" - Similarity:           {:.2%}\n".format(similarity(mode)))

# Classificatore `AdaBoost`
Un classificatore AdaBoost è un meta-stimatore che inizia inserendo un classificatore nel set di dati originale e quindi adatta copie aggiuntive del classificatore nello stesso set di dati, ma dove i pesi delle istanze classificate in modo errato vengono regolati in modo tale che i classificatori successivi si concentrino maggiormente sui casi difficili.

In [13]:
clf = AdaBoostClassifier(n_estimators = 500, algorithm = "SAMME", random_state = 0)
test_boosting(clf)
test_boosting(clf, MinMaxScaler(feature_range = (0, 1)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results:
 - Mean cross_val_score: 83.83%
 - Mean score:           85.98%
 - Similarity:           91.67%



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results (scaled tasks):
 - Mean cross_val_score: 85.66%
 - Mean score:           88.64%
 - Similarity:           91.67%



# Classificatore Gradient Boosting

Questo algoritmo costruisce un modello additivo in modo graduale; consente l'ottimizzazione di funzioni di perdita differenziabili arbitrarie. In ogni fase, `n_classes_` alberi di regressione sono adattati al gradiente negativo della funzione di perdita, ad es. perdita di log binaria o multiclasse.

In [14]:
clf = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.01, random_state = 0)
test_boosting(clf)
test_boosting(clf, MinMaxScaler(feature_range = (0, 1)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results:
 - Mean cross_val_score: 83.75%
 - Mean score:           89.02%
 - Similarity:           91.67%



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results (scaled tasks):
 - Mean cross_val_score: 86.06%
 - Mean score:           87.12%
 - Similarity:           91.67%



# Gradient Boosting di albero di classificazione basato sugli istogrammi
Durante l'addestramento, il coltivatore di alberi impara ad ogni punto di divisione se i campioni con valori mancanti devono andare al figlio sinistro o destro, in base al potenziale guadagno. Durante la previsione, i campioni con valori mancanti vengono assegnati di conseguenza al figlio sinistro o destro. Se durante l'addestramento non sono stati rilevati valori mancanti per una determinata funzionalità, i campioni con valori mancanti vengono mappati a qualunque figlio abbia il maggior numero di campioni.

In [15]:
clf = HistGradientBoostingClassifier(max_iter = 500, learning_rate = 0.01, random_state = 0)
test_boosting(clf)
test_boosting(clf, MinMaxScaler(feature_range = (0, 1)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results:
 - Mean cross_val_score: 83.80%
 - Mean score:           89.77%
 - Similarity:           95.83%



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results (scaled tasks):
 - Mean cross_val_score: 83.80%
 - Mean score:           89.77%
 - Similarity:           95.83%



# Classificatore Percettrone multi-strato
Vengono scalati i task tramite uno `StandardScaler`, in seguito viene eseguita una ricerca sulla [griglia di parametri](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#tips-on-practical-use), e viene testato su tutti i Task un `MLPCLassifier` con i parametri migliori.

In [16]:
sc = StandardScaler()

scaled_tasks = copy.deepcopy(tasks)

for task in scaled_tasks:
  task.scale(sc)

param_grid = {
  "hidden_layer_sizes": [(150, 100, 50), (120, 80, 40), (100, 50, 30)],
  "activation": ['identity', 'logistic', 'tanh', 'relu'],
  "alpha": 10.0 ** -np.arange(1, 7),
  "max_iter": [1000, 2000, 3000]
}
search = HalvingGridSearchCV(MLPClassifier(solver = "lbfgs", random_state = 0), param_grid, cv = 4, n_jobs = -1)

cross_val_scores = []
scores = []
predictions = []

for task in scaled_tasks:
  search.fit(task.X_train, task.y_train)

  clf = MLPClassifier(**search.best_params_, solver = "lbfgs", random_state = 0)

  cross_val_scores.append(cross_val_score(clf, task.X_train, task.y_train, cv = 4, n_jobs = -1).mean())
  clf.fit(task.X_train, task.y_train)
  scores.append(clf.score(task.X_test, task.y_test))
  predictions.append(clf.predict(task.X_test))

print("Results (scaled tasks):")
print(" - Mean cross_val_score: {:.2%}".format(np.mean(cross_val_scores)))
print(" - Mean score:           {:.2%}".format(np.mean(scores)))

mode, count = stats.mode(predictions)
print(" - Similarity:           {:.2%}\n".format(similarity(mode)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklea

Results (scaled tasks):
 - Mean cross_val_score: 76.03%
 - Mean score:           77.65%
 - Similarity:           91.67%

